# <div align="center">Programa Nacional de Vigilância em Saúde em Navios de Cruzeiro</div>

<div align="center"><img src="https://i.imgur.com/FkLJ24t.png"></div>

A análise visa responder algumas perguntas sobre inspeção de navios de cruzeiro que faz parte do **Programa Nacional de Vigilância Sanitária para Navios de Cruzeiro** realizado pela **Anvisa** no perído de outubro até o mês de abril do ano subsquente. O objetivo deste programa é avaliar a segurança sanitária dos alimentos e da água para consumo nas embarcações além de acompanhar o Plano de Limpeza e Desinfecção nas instalações e áreas internas dos navios reforçando a prevenção de surtos de doenças no decorrer das viagens.

A Anvisa classifica os navios em níveis:

|Nível |Pontuação de Risco | Categoria |
|:---:|:---:|:---:|
| A | 0-150 | Excelente|
| B | 151-300 | Satisfatório|
| C | 301-450 | Aceitável|
| D | 450-... | Insatisfatório|

<br>

Nesta análise são mostradas as características de **selecão, ordenamento e filtragem** da linguagem **SQL**

**Dataset em formato .CSV está disponível:**
- [Anvisa](https://dados.anvisa.gov.br/dados/DADOS_ABERTOS_INSPECAO_NAVIO.csv)
- [Projeto](./DADOS_ABERTOS_INSPECAO_NAVIO.csv).

**Dicionário de dados em formato .PDF está disponível**
- [Anvisa](https://dados.anvisa.gov.br/dados/Documentacao_e_Dicionario_de_Dados_Navios_Cruzeiro.pdf)
- [Projeto](./Documentacao_e_Dicionario_de_Dados_Navios_Cruzeiro.pdf)

<hr>

## <a name="indice"></a> ÍNDICE

**Obs**: direcionamento do índice apenas quando este notebook é executado pelo Jupyter Notebook.

- [Preparação do Ambiente](#Preparação_Ambiente)
- [Análise](#analise)
    1. [Total de Embarcações Distintas](#1)
    2. [Número Total de Inspeções](#2)
    3. [Quais embarcações e quando ocorreu determinada classificação de risco para uma embarcação](#3)
    4. [Navios com avaliação de qualidade máxima](#4)
    5. [As 10 piores ocorrências de inspeção](#5)
    6. [Meios: Navios com padrões B e C e pontuação de risco no range de 151 a 450](#6)
    7. [Extremos: Navios com padrões A e D e pontuação de risco FORA do range de 151 a 450](#7)
    8. [COVID-19: Embarcações com iniciais A e B, índice de conformidade B ou C e conformidade <= 90% antes da pandemia](#8)

<hr>

## <a name="Preparação_Ambiente"></a> Preparação do Ambiente

In [ ]:
%pip install sqlalchemy
%pip install ipython-sql

In [ ]:
# Bibliotecas
from sqlalchemy import create_engine
import sqlite3
import csv

# Conexão SQLite3: cria o banco de dados anvisa.db
conexao = sqlite3.connect("anvisa.db")
cursor = conexao.cursor()

# Conexão ipython-sql - execução SQL em tempo de execução
%reload_ext sql
%sql sqlite+pysqlite:///anvisa.db



In [ ]:
%%sql

/*Criação e estrutura da tabela tb_navios*/

CREATE TABLE IF NOT EXISTS tb_navios(
  nome_navio VARCHAR(50) NULL,
  mes_ano VARCHAR(10) NULL,
  classificacao_risco VARCHAR(15) NULL,
  indice_conformidade VARCHAR(15) NULL,
  pontuacao_risco INT NULL,
  temporada VARCHAR(256) NULL);

In [ ]:
# Extração dos dados e inserção na tabela tb_navios

with open("./DADOS_ABERTOS_INSPECAO_NAVIO.csv","r") as dataset:
    anvisa_dataset = csv.reader(dataset, delimiter=";")
    cursor.executemany("Insert into tb_navios values (?, ?, ?, ?, ?, ?)", anvisa_dataset)
    conexao.commit()

<hr>

## <a name="analise"></a> Análise

1 - Inicialmente podemos contabilizar quantas embarcações diferentes foram inspecionadas ao longo de todo registro de inspeções. Em todo o período, **132 embarcações distintas passaram pela inspeção.**

[Voltar ao início](#indice)

In [40]:
%%sql 

SELECT 
    COUNT(DISTINCT nome_navio) AS 'Quantidade de Navios Inspecionados'
FROM
    tb_navios;

 * sqlite+pysqlite:///anvisa.db
Done.


Quantidade de Navios Inspecionados
132


2. Contabilizar o **número total de inspeções ao longo do período**. No total foram **463 inspeções**.

[Voltar ao início](#indice)

In [41]:
%%sql

SELECT 
    COUNT(*) AS 'Quantidade Total de Inspeções'
FROM
    tb_navios;

 * sqlite+pysqlite:///anvisa.db
Done.


Quantidade Total de Inspeções
463


3. Neste dataset, várias embarcações foram inspecionadas mais de uma vez. **Podemos saber quais navios, pelo menos uma vez, receberam uma determinada classificação de risco e quando isso ocorreu.**

[Voltar ao início](#indice)

In [43]:
%%sql

SELECT DISTINCT
    nome_navio AS 'Nome do Navio',
    classificacao_risco AS 'Classificação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    classificacao_risco = 'A'
ORDER BY nome_navio ASC , mes_ano DESC
LIMIT 20;

 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Classificação de Risco,Data da Inspeção
ADONIA,A,12/2017
ADONIA,A,03/2016
ADONIA,A,02/2012
ADONIA,A,01/2013
AIDA AURA,A,11/2019
AIDA CARA,A,11/2011
AIDA CARA,A,03/2014
AIDA CARA,A,02/2012
AIDA VITA,A,12/2012
AIDA VITA,A,06/2011


4. **Quais ocasiões as embarcações conseguiram alcançar os requisitos plenos da Anvisa, ou seja, índice de conformidade de 100% e pontuação de risco 0 pelo menos uma vez.** Como os resultados de índice e pontuação serão idênticos, podemos ordenar pelo nome do navio na ordem A-Z (ou crescente) e a data de ocorrência da inspeção. Neste caso **nenhum navio conseguiu tal feito**.

[Voltar ao início](#indice)

In [46]:
%%sql
SELECT 
    nome_navio AS 'Nome do Navio',
    indice_conformidade AS 'Índice de Conformidade',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    indice_conformidade = 100
        AND pontuacao_risco = 0
ORDER BY mes_ano ASC, nome_navio ASC;

 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Índice de Conformidade,Pontuação de Risco,Data da Inspeção


5. **Embarcações com os piores indicadores?** Selecionamos os 10 piores resultados da classificaçao D (acima de 450 pontos) juntamente com a data da ocorrência.

[Voltar ao início](#indice)

In [47]:
%%sql

SELECT DISTINCT
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco > 450
ORDER BY pontuacao_risco DESC
LIMIT 10;


 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Pontuação de Risco,Data da Inspeção
SEA EXPLORER,1230,03/2014
STAR PRINCESS,1115,01/2012
ORIENT QUEEN II,1065,12/2013
SOVEREIGN (SOBERANO),1035,12/2012
OCEAN DREAM,964,10/2011
SEVEN SEAS MARINER,875,03/2011
MIDNATSOL,831,10/2016
CRYSTAL SERENITY,830,03/2013
MSC MUSICA,805,03/2012
MAASDAM,790,02/2013


6. Ao retornar valores em um intervalo determinado podemos **selecionar as embarcações que receberam padrões B e C pelo menos uma vez retornando um range de 151 até 450 pontos de risco** que correspondem a tais classificações.

[Voltar ao início](#indice)

In [48]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco BETWEEN 151 AND 450
ORDER BY nome_navio ASC , mes_ano DESC;

 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Pontuação de Risco,Data da Inspeção
AIDA AURA,160,10/2018
AIDA CARA,360,12/2010
AIDA CARA,310,11/2013
AIDA CARA,260,11/2012
AIDA CARA,255,10/2017
AIDA VITA,195,10/2010
AMADEA,231,11/2012
AMADEA,430,02/2017
AMADEA,240,01/2016
AMERA (PRINSENDAM),200,12/2015


7. **O inverso também se aplica caso não necessitemos que determinado grupo seja selecionado**. Agora vamos retornar apenas os navios que já estiveram no padrão A ou D

[Voltar ao início](#indice)

In [50]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    pontuacao_risco AS 'Pontuação de Risco',
    mes_ano AS 'Data da Inspeção'
FROM
    tb_navios
WHERE
    pontuacao_risco NOT BETWEEN 151 AND 450
ORDER BY pontuacao_risco , nome_navio , mes_ano ASC;

 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Pontuação de Risco,Data da Inspeção
ADONIA,0,01/2013
ADONIA,0,03/2016
ADONIA,0,12/2017
AIDA CARA,0,02/2012
AIDA VITA,0,02/2012
AIDA VITA,0,02/2013
AIDA VITA,0,03/2012
AIDA VITA,0,06/2011
AIDA VITA,0,12/2012
ALBATROS,0,01/2014


8. Embarcações que **comecem com a letra 'A' ou 'B' que já receberam classificação, pelo menos uma vez, dos níveis B ou C e tiveram índices de conformidade menor ou igual a 95% em inspeções antes do ano em que a pandemia do COVID-19 chegou ao Brasil (2020)?**

[Voltar ao início](#indice)

In [53]:
%%sql

SELECT 
    nome_navio AS 'Nome do Navio',
    mes_ano AS 'Data da Inspeção',
    classificacao_risco AS 'Classificação de Risco',
    indice_conformidade AS 'Ìndice de Conformidade',
    temporada AS Temporada
FROM
    tb_navios
WHERE
    (classificacao_risco = 'B'
        OR classificacao_risco = 'C')
        AND indice_conformidade <= 95
        AND mes_ano < '01/2020'
        AND (nome_navio LIKE 'A%'
        OR nome_navio LIKE 'B%')
ORDER BY mes_ano , indice_conformidade , nome_navio;

 * sqlite+pysqlite:///anvisa.db
Done.


Nome do Navio,Data da Inspeção,Classificação de Risco,Ìndice de Conformidade,Temporada
AURORA,01/2011,C,"86,95",Programa de Inspecao Navios de Cruzeiro 2010/2011 - Janeiro 2011
BLACK WATCH,01/2014,C,"91,84",Programa de Inspecao Navios de Cruzeiro 2013/2014 - Janeiro 2014
ARCADIA,01/2016,B,"94,00",Programa de Inspecao Navios de Cruzeiro 2015/2016 - Janeiro 2016


In [52]:
#Fecha conexão com o banco anvisa.db

conexao.close()